In [1]:
import os
from datetime import datetime, date
import tarfile
import requests
import wradlib as wrl
import re
import matplotlib.pyplot as plt

import numpy as np
import osr

import pathlib

In [2]:
RADOLAN_FTP_URL = "https://opendata.dwd.de/climate_environment/CDC/grids_germany/daily/radolan/historical/bin/2017/SF201707.tar.gz"

START_DATE = datetime(2017, 7, 24)
END_DATE = datetime(2017, 7, 27)

GEOTIFF_OUTPUT_FOLDER = "./gtiff/"
pathlib.Path(GEOTIFF_OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

In [3]:
# download archive
r = requests.get(RADOLAN_FTP_URL, stream=True)
with open("output.tar.gz", "wb") as output:
    output.write(r.content)
    
# extract archive into temp
tar = tarfile.open("output.tar.gz", "r:gz")
tar.extractall(path="/tmp/radolan")
tar.close()

print("Successfully downloaded")

Successfully downloaded


In [4]:
name_regex='raa01-sf_10000-(.+?)-dwd---bin'
date_format='%y%m%d%H%M'

def datetime_is_between(target, dt_start, dt_end):
    if dt_start is not None and dt_end is not None:
        return dt_start <= target <= dt_end
    elif dt_start is not None:
        return dt_start <= target
    elif dt_end is not None:
        return target <= dt_end
    else:
        return True
    
def parse_date(filename):
    match = re.search(name_regex, filename)
    if match:
        found = match.group(1)
        result_dt = datetime.strptime(found, date_format)
        return result_dt
    return None

files = [os.path.join("/tmp/radolan", f) for f in os.listdir("/tmp/radolan/") if datetime_is_between(parse_date(f), START_DATE, END_DATE)]
files.sort(key=lambda filename: parse_date(filename))
print(f"Found {len(files)} radolan files")

Found 72 radolan files


In [5]:
# function for reading radolan files and removing secondary
def read_radolan(filename):
    composite, attrs = wrl.io.radolan.read_radolan_composite(filename)
    composite = np.float32(composite)

    datetime = attrs['datetime']
    producttype = attrs['producttype']
    nodataflag  = attrs['nodataflag']
    secondary = attrs['secondary']

    if secondary is not None:
        composite.flat[secondary] = nodataflag
        composite = np.ma.masked_equal(composite, -9999)

    return datetime, composite

In [6]:
# create projections
proj_stereo = wrl.georef.create_osr("dwd-radolan")
proj_wgs = osr.SpatialReference()
proj_wgs.ImportFromEPSG(4326)
proj_utm32 = osr.SpatialReference()
proj_utm32.ImportFromEPSG(32632)

radolan_grid_xy = wrl.georef.get_radolan_grid(900,900)
radolan_grid_ll = wrl.georef.reproject(radolan_grid_xy, projection_source=proj_stereo, projection_target=proj_wgs)
radolan_grid_utm32 = wrl.georef.reproject(radolan_grid_ll, projection_source=proj_wgs, projection_target=proj_utm32)

In [7]:
# write as geotiff WGS84
for filename in files:
    datetime, composite = read_radolan(filename)
    
    reduced = np.array(composite, copy=True)
    reduced[reduced > 0.0] = reduced[reduced > 0.0] * 10
    reduced = reduced.astype(np.int16)

    data, xy = wrl.georef.set_raster_origin(reduced, radolan_grid_utm32, 'upper')
    ds = wrl.georef.create_raster_dataset(data, xy, projection=proj_utm32, nodata=-9999)

    filename = os.path.join(GEOTIFF_OUTPUT_FOLDER, filename + ".tif")
    wrl.io.write_raster_dataset(filename, ds, 'GTiff')

In [8]:
import imageio

# Get coordinates
radolan_grid_xy = wrl.georef.get_radolan_grid(900,900)
x = radolan_grid_xy[:,:,0]
y = radolan_grid_xy[:,:,1]


def plot_radolan(data, clabel=None):
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111, aspect='equal')
    pm = ax.pcolormesh(x, y, data, vmin=0, vmax=75, cmap='viridis')
    cb = fig.colorbar(pm, shrink=0.75)
    cb.set_label(clabel)
    plt.xlabel("x [km]")
    plt.ylabel("y [km]")
    plt.xlim((x[0,0],x[-1,-1]))
    plt.ylim((y[0,0],y[-1,-1]))
    
    fig.canvas.draw()
    fig.canvas.flush_events()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close()
    return image
 
images = [plot_radolan(read_radolan(radfile)[1]) for radfile in files]
imageio.mimsave("output.gif", images, format='GIF', fps=5)

In [9]:
from IPython import display
display.HTML('<img src="{}">'.format("output.gif"))